## TempAlert Automation for GitHub

Using in Github workflow for automation

In [2]:
# Install Selenium
!pip install selenium -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 50.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 23.6 MB/s eta 0:00:00


In [3]:
!apt-get update -qq
!apt-get install -y chromium-chromedriver -qq
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Preconfiguring packages ...
(Reading database ... 127400 files and directories currently installed.)
Preparing to unpack .../systemd-sysv_249.11-0ubuntu3.15_amd64.deb ...
Unpacking systemd-sysv (249.11-0ubuntu3.15) over (249.11-0ubuntu3.12) ...
Preparing to unpack .../libpam-systemd_249.11-0ubuntu3.15_amd64.deb ...
Unpacking libpam-systemd:amd64 (249.11-0ubuntu3.15) over (249.11-0ubuntu3.12) ...
Preparing to unpack .../systemd_249.11-0ubuntu3.15_amd64.deb ...
Unpacking systemd (249.11-0ubuntu3.15) over (249.11-0ubuntu3.12) ...
Preparing to unpack .../libsystemd0_249.11-0ubuntu3.15_amd64.deb ...
Unpacking libsystemd0:amd64 (249.11-0ubuntu3.15) over (249.11-0ubuntu3.12) ...
Setting up libsystemd0:amd64 (249.11-0ubuntu3.15) ...
Selecting previously unselected package apparmor.
(Reading database ... 1274

In [4]:
!which chromium-browser
!which chromedriver

/usr/bin/chromium-browser
/usr/bin/chromedriver


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service

import time
from selenium.webdriver.common.keys import Keys

import os
#from kaggle_secrets import UserSecretsClient

# Configure Chrome options for Kaggle's headless environment
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--headless=new") 
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.binary_location = "/usr/bin/chromium-browser"

### Using Github Secrets

In [6]:
temp_pass =os.getenv("TEMPALERT_PASSWORD")
temp_user = os.getenv("TEMPALERT_USER")
smtp_password = os.getenv("EMAIL_SMTP")
server_login = os.getenv('EMAIL_SERVER')

In [7]:
print(f"TEMPALERT_USER loaded: {temp_user}")
print(f"TEMPALERT_PASSWORD loaded? {'YES' if temp_pass else 'NO'}")

TEMPALERT_USER loaded: None
TEMPALERT_PASSWORD loaded? NO


In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://app.smartsense.co/")

sensors = ["✅ SmartSense Automation Report\n"]

# === STEP_1: Login
for field, value in [("userName", temp_user), ("password", temp_pass)]:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, field))).send_keys(value)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))).click()

print("✅ Logged in")

# === STEP_2A Function to extract final temperature based on sensor name
def get_final_temp(sensor_name):
    sensor = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'asset-item-header') and contains(., '{sensor_name}')]"))
    ).find_element(By.XPATH, "./following-sibling::div")
        
    return sensor.find_element(By.XPATH, ".//div[contains(@class, 'asset-widget-chart__reading-container')]/span").text

# === STEP_2B Extract and print temperatures for all three sensors
for sensor_name in ["RX Freezer 1", "RX Ambient 1", "RX Cooler 1"]:
    print(f"{sensor_name}: {get_final_temp(sensor_name)}")
    sensors.append((sensor_name, get_final_temp(sensor_name)))

# === STEP_2C Function to check checklist status
def check_list_status(checklist_time):
    try:
        checklist = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'dashboard-daily-checklist__time') and contains(., '{checklist_time}')]/../div[contains(@class, 'dashboard-daily-checklist__footer')]/i"))
        ).text
        return checklist if checklist else "NOT STARTED"
    except:
        return "NOT STARTED"

# === STEP_2D Check AM and PM Daily Checklists
am_checklist = check_list_status('12:00 AM - 12:00 PM')
pm_checklist = check_list_status('12:00 PM - 12:00 AM')
print(am_checklist)
print(pm_checklist)
sensors.append(f'AM checklist - {am_checklist}')
sensors.append(f'PM checklist - {pm_checklist}')


if am_checklist == 'COMPLETED' and pm_checklist == 'COMPLETED':
    print('All tasks completed')
    sensors.append('All Tasks Completed!')
else:
    

    # === STEP 3: Click the COMPLETE button (known to work) ===
    complete_btn = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//button[span[normalize-space()='COMPLETE']]"))
    )
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", complete_btn)
    time.sleep(1)
    
    span_elem = complete_btn.find_element(By.TAG_NAME, "span")
    driver.execute_script("""
      const ev = new MouseEvent('click', {
        view: window,
        bubbles: true,
        cancelable: true
      });
      arguments[0].dispatchEvent(ev);
    """, span_elem)
    
    print("✅ COMPLETE button clicked (React-style)")
    sensors.append("✅ COMPLETE button clicked (React-style)")
    driver.save_screenshot("after_complete_click.png")

    # === STEP 4: Wait for checkboxes to load ===
    time.sleep(4)  # wait for page transition
    
    # === STEP 5: Wait for checkboxes to load ===
    time.sleep(4)  # allow page to load fully
    
    # === STEP 6: Click and confirm each checkbox ===
    checkboxes = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "input.ant-checkbox-input"))
    )
    
    print(f"🔘 Found {len(checkboxes)} checkboxes")
    sensors.append(f"🔘 Found {len(checkboxes)} checkboxes")
    
    # Only proceed with the first 3
    for i in range(min(3, len(checkboxes))):
        checkbox = checkboxes[i]
        
        try:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", checkbox)
            time.sleep(0.5)
    
            # Click with MouseEvent
            driver.execute_script("""
              const evt = new MouseEvent('click', {
                view: window,
                bubbles: true,
                cancelable: true
              });
              arguments[0].dispatchEvent(evt);
            """, checkbox)
    
            # Confirm selection
            is_checked = checkbox.is_selected()
            parent = checkbox.find_element(By.XPATH, "..")
            checkbox_class = parent.get_attribute("class")
    
            print(f"✅ Checkbox {i+1} clicked")
            sensors.append(f"✅ Checkbox {i+1} clicked")
            print(f"☑️ Checkbox {i+1} is_selected(): {is_checked}")
            sensors.append(f"☑️ Checkbox {i+1} is_selected(): {is_checked}")
            print(f"👀 Checkbox {i+1} parent class: {checkbox_class}")
            sensors.append(f"👀 Checkbox {i+1} parent class: {checkbox_class}")
    
        except Exception as e:
            print(f"❌ Failed to click or check checkbox {i+1}: {e}")
            sensors.append(f"❌ Failed to click or check checkbox {i+1}: {e}")

    # Save final screenshot
    driver.save_screenshot("all_3_checkboxes_checked.png")
    
    # === STEP 7: Click the final "Complete" button ===
    try:
        complete_final_btn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[span[normalize-space()='Complete']]"))
        )
    
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", complete_final_btn)
        time.sleep(0.5)
    
        # Click the inner <span> to trigger React
        span_elem = complete_final_btn.find_element(By.TAG_NAME, "span")
        driver.execute_script("""
          const ev = new MouseEvent('click', {
            view: window,
            bubbles: true,
            cancelable: true
          });
          arguments[0].dispatchEvent(ev);
        """, span_elem)
    
        print("✅ Final COMPLETE button clicked")
        sensors.append("✅ Final COMPLETE button clicked")
        driver.save_screenshot("final_submit_clicked.png")
    
    except Exception as e:
        print("❌ Failed to click final COMPLETE button:", e)
        sensors.append("❌ Failed to click final COMPLETE button:", e)
        driver.save_screenshot("final_submit_error.png")

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from_email = "amrit@pillview.com"  # Same as SMTP login
to_email = "avirdee@aol.com"    # Or a phone SMS gateway
smtp_password = smtp_password
server_login = server_login

In [ ]:
# Compose message
msg = MIMEMultipart()
msg["From"] = from_email
msg["To"] = to_email
msg["Subject"] = "TempAlert Info"

# Create the email body
body = "Hi there,\n\nHere is the current TempAlert info:\n\n"
report_lines = []

for item in sensors:
    if isinstance(item, tuple):
        report_lines.append(f"{item[0]}: {item[1]}")
    else:
        report_lines.append(str(item))  # Handles strings like 'COMPLETED'

# Join with line breaks
body += "\n".join(report_lines)
body += "\n\nRegards,\nTempBot"

# Add to email
msg.attach(MIMEText(body))

# Send email
with smtplib.SMTP("smtp-relay.brevo.com", 587) as server:
    server.starttls()
    server.login(server_login, smtp_password)
    server.send_message(msg)

print("✅ Email sent!")
print(msg)